
# **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


*Daniel Edgardo García Bonilla*

## 💡 ¿Qué es la reducción de dimensionalidad?

La **reducción de dimensionalidad** es un proceso mediante el cual transformamos un conjunto de datos que tiene **muchas variables (dimensiones)** en otro conjunto con **menos variables**, conservando la mayor cantidad posible de la información relevante.

Imagina que tienes una matriz de utilidad (usuarios vs. productos) con miles de columnas (por ejemplo, películas o restaurantes). Esa matriz es:

- **Muy grande**
- **Dispersa** (la mayoría de los usuarios no califican todos los productos, así que hay muchos valores faltantes)

La reducción de dimensionalidad nos ayuda a **resumir** esa información en un espacio más pequeño que capture los patrones más importantes.



## 🎯 ¿Por qué se utiliza en algoritmos de recomendación?

En los sistemas de recomendación, como los basados en **filtrado colaborativo**, el objetivo es encontrar relaciones ocultas entre usuarios y productos. Aquí es donde entra la reducción de dimensionalidad (por ejemplo, mediante **SVD — Descomposición en Valores Singulares**):

✅ **Identifica patrones latentes:**  
Reduce el tamaño del problema al detectar factores ocultos o *variables latentes* que explican las preferencias (por ejemplo: un usuario podría preferir restaurantes con buen servicio o películas de cierto género, aunque eso no se vea explícitamente en los datos).

✅ **Mejora la eficiencia:**  
Al trabajar con un número menor de dimensiones, los cálculos son más rápidos y el sistema es menos costoso computacionalmente.

✅ **Generaliza mejor:**  
Al eliminar ruido y redundancias, se pueden hacer recomendaciones más robustas y precisas, evitando el sobreajuste a calificaciones específicas.

✅ **Maneja mejor la dispersión:**  
En una matriz con muchos datos faltantes, la reducción de dimensionalidad permite "rellenar" de forma inteligente las partes que no conocemos, basándose en los patrones descubiertos.



In [1]:
# Incluye aquí todas las librerías y paquetes que requieras.
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

### 📈**Dataset Analizado.**
Para este ejercicio exploraremos la data de calificaciones de restaurantes extraída de ICS. De ahí necesitaremos los archivos **"rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [2]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)



El uso de Latin-1 resuelve conflictos cuando el formato del DataFrame no está en UTF-8 y contiene elementos que está por fuera del alfabeto inglés como ser, el Español y el Francés. Al ser nombres de diferentes tipos de restaurantes, estoy usando este argumento para poder leer de manera correcta el archivo

In [3]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [4]:
# Del segundo archivo obtenemos información diversa de cada restaurante:
data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [5]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

In [6]:
# Definimos el DataFrame que conjunta la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":


df_combinado = pd.merge(data1a, data2a, on='placeID')

# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1077,135038,2,1,Restaurant la Chalita
2,U1077,132825,2,2,puesto de tacos
3,U1077,135060,2,2,Restaurante Marisco Sam
4,U1068,135104,1,2,vips


In [7]:
#    Definimos la matriz de utilidad cuyos renglones son los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".

UtMx_food = df_combinado.pivot_table(values='food_rating', columns='userID', index='name', fill_value=0)

print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [8]:
# a. Aplicamos la transformación SVD para obtener la matriz de las variables
#    latentes de los restaurantes, en relación a la manera en que los usuarios 
#    evaluaron la comida. 
nc_food = 138     # número de componentes

print('Total de valores singulares basados en evaluación de la comida:', nc_food)

Total de valores singulares basados en evaluación de la comida: 138


#### 🍜**Mejores Evaluaciones Respecto al Restaurante**

A manera de ejemplo, se tomará un restaurante de referencia, para en base a este recomendar otros restaurantes a un cliente.


In [9]:
# b. Llevamos a cabo la factorización SVD truncada en la matriz de utilidad
#    obtenida previamente y con respecto al número de componentes dada 
#    por la variable nc_food.

SVD_food = TruncatedSVD(n_components=nc_food, random_state=1)
SVD_food.fit(UtMx_food)


# Con la factorización SVD truncada obtenida, determinemos la cantidad 
# de componentes que explican un 90% de la variabilidad acumulada de 
# cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1


print('Valor del umbral o truncamiento al 90% de dicha variabilidad:', N_food)

Valor del umbral o truncamiento al 90% de dicha variabilidad: 51


In [10]:
# Usando la métrica de correlación de Pearson, obtenemos ahora las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similitudes con el "Restaurante Pueblo Bonito" 
# y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtenemos las 10 mejores correlaciones positivas a continuación.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:5]

Total de similaridades positivas encontradas: 65
Algunos de los resultados encontrados:


[(np.float64(0.041392920998984215), 'Cabana Huasteca'),
 (np.float64(0.01709006482903871), 'Cafe Chaires'),
 (np.float64(0.009526057087048694), 'Cafeteria cenidet'),
 (np.float64(0.24459809652208758), 'Cafeteria y Restaurant El Pacifico'),
 (np.float64(0.023194124763133095), 'Cenaduria El RincÃ³n de Tlaquepaque')]

In [11]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente. Llamaremos "mejores_sim_food_ordenadas" a 
# dicha lista.

mejores_sim_food_ordenadas = sorted(mejores_sim_food, reverse=True)


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (np.float64(0.7910409754989735), 'Restaurante la Estrella de Dima')
2> (np.float64(0.704815985120805), 'pizza clasica')
3> (np.float64(0.6647277830142064), 'Restaurante Guerra')
4> (np.float64(0.5483206980941779), 'El Club')
5> (np.float64(0.38674934236723996), 'Pizzeria Julios')
6> (np.float64(0.3198002367708535), 'Hamburguesas saul')
7> (np.float64(0.3033401789251), 'Restaurant Oriental Express')
8> (np.float64(0.2990234595215336), 'la Cochinita Pibil Restaurante Yucateco')
9> (np.float64(0.29125529791748006), 'Restaurante El Cielo Potosino')
10> (np.float64(0.28792999069955805), 'La Fontana Pizza Restaurante and Cafe')


### 👨‍🍳**Mejores evaluaciones respecto al servicio.**

De manera análoga y usando ahora la evaluación de un restaurante con respecto a su servicio (service_rating), encuentraremos ahora los diez restaurantes con mayor similaridad al mismo restaurante anterior llamado "Restaurante Pueblo Bonito" y con respecto también a la misma métrica de similaridad de Pearson.



In [12]:
# a. Definimos la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


UtMx_service = df_combinado.pivot_table(values='service_rating', columns='userID', index='name', fill_value=0)


print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
nc_service = 138


SVD_service = TruncatedSVD(n_components=nc_service, random_state=1)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

In [14]:
SVD_service = TruncatedSVD(n_components=nc_service, random_state=1)
SVD_service.fit(UtMx_service)

for j in range(nc_service):
    if SVD_service.explained_variance_ratio_[0:j].sum() > 0.90:
        break
    
N_service = j - 1
print('Valor del truncamiento al 90% de dicha variabilidad:', N_service)

Valor del truncamiento al 90% de dicha variabilidad: 51


In [15]:
corr_mat_service = np.corrcoef(resultant_matrix_service)

#vector de correlacion del restaurante en la matriz de correlacion respecto al dataset de servicio
nombres_rest = UtMx_service.T.columns
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_service[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_service = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_service.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_service))



Total de similaridades positivas encontradas: 56


In [16]:
mejores_sim_service_ordenadas = sorted(mejores_sim_service, reverse=True)

# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de servicio con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_service_ordenadas[k]))

Similitudes con base a la evaluación de servicio con mayores valores de correlación:
1> (np.float64(0.5751351005541153), 'pizza clasica')
2> (np.float64(0.5201427943851279), 'El Club')
3> (np.float64(0.4390437503854505), 'Restaurante la Estrella de Dima')
4> (np.float64(0.40146983411969217), 'Restaurante Guerra')
5> (np.float64(0.35779572734705417), 'Restaurant Orizatlan')
6> (np.float64(0.3376744053004511), 'la Cochinita Pibil Restaurante Yucateco')
7> (np.float64(0.3362598126633021), 'Restaurante El Cielo Potosino')
8> (np.float64(0.2919651121138544), 'puesto de tacos')
9> (np.float64(0.28170207743421766), 'Restaurant Oriental Express')
10> (np.float64(0.2805268511299617), 'Hamburguesas saul')


In [17]:
#Encuentro las 10 mejores recomendaciones basadas en la comida y el servicio:
mejores_10_comida, mejores_10_servicio = [mejores_sim_food_ordenadas[k][1] for k in range(1,11)], [mejores_sim_service_ordenadas[k][1] for k in range(1,11)]
mejores_recomendaciones = [restaurante for restaurante in mejores_10_comida if restaurante in mejores_10_servicio]
print('Recomendaciones basadas en la comida y el servicio:')
mejores_recomendaciones

Recomendaciones basadas en la comida y el servicio:


['Restaurante la Estrella de Dima',
 'pizza clasica',
 'Restaurante Guerra',
 'El Club',
 'Hamburguesas saul',
 'Restaurant Oriental Express',
 'la Cochinita Pibil Restaurante Yucateco',
 'Restaurante El Cielo Potosino']

## 📌 **Conclusiones**
1) De acuerdo con los resultados obtenidos se concluye que para una persona que haya visitado el restaurante de referencia (Restaurante Pueblo Bonito), el sistema encontró una alta probabilidad de que también a esta persona le gusten los siguientes restaurantes:

    * Restaurante la Estrella de Dima
    * pizza clasica
    * Restaurante Guerra
    * El Club
    * Hamburguesas saul
    * Restaurant Oriental Express
    * la Cochinita Pibil Restaurante Yucateco
    * Restaurante El Cielo Potosino

2) SVD es una herramienta matemática que permite descomponer la compleja matriz de interacciones usuario-ítem en sus componentes fundamentales (preferencias de usuario, características de ítem y su importancia), permitiendo así predecir de forma muy efectiva las calificaciones faltantes y generar recomendaciones personalizadas.

3) A diferencia del Análisis de Componentes Principales (PCA), las implementaciones de SVD como TruncatedSVD no requieren centrar los datos antes de la descomposición. Esta distinción es fundamental para su eficiencia en el manejo de matrices dispersas.
PCA necesita establecer un punto de referencia (la media) para medir la varianza, lo que implica convertir los ceros —que a menudo significan "ausencia de datos"— en valores numéricos. Este proceso destruye la dispersión de la matriz, provocando un uso masivo de memoria y alterando el significado original de los datos.
Al omitir el centrado, SVD opera directamente sobre la matriz original, preservando su estructura dispersa. Esto le permite procesar grandes volúmenes de datos de manera eficiente, manteniendo la integridad de la información y convirtiéndolo en la opción ideal para aplicaciones como sistemas de recomendación y análisis de texto.
